# Tareas

## Tarea 1

Detección de colores de la ropa y de si es hombre o mujer con un modelo (enrealidad 2) entrenado por nosotros

### Importaciones

In [ ]:
import os
import numpy as np
from skimage import io, color
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import kagglehub
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPUs detectadas:", tf.config.list_physical_devices('GPU'))

2025-11-13 18:43:06.271845: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.20.0
GPUs detectadas: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Definición de datasets

In [ ]:
dataset_Gender = "DatabaseGender"
dataset_emotions = kagglehub.dataset_download("jonathanoheix/face-expression-recognition-dataset")

Ruta dataset_Gender: DatabaseGender
Ruta dataset_emotions: /home/nicolo/.cache/kagglehub/datasets/jonathanoheix/face-expression-recognition-dataset/versions/1
Contenido de dataset_emotions: ['images']


### Cargar datasets

In [5]:
def load_dataset(path, image_size=(48, 48)):
    X, y = [], []
    labels = os.listdir(path)
    for label in labels:
        folder = os.path.join(path, label)
        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, image_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

X_gender, y_gender = load_dataset(dataset_Gender, (64,64))
X_emotion, y_emotion = load_dataset(
    os.path.join(dataset_emotions, "images/train"), (48,48)
)



### Normalizar y Etiquetar

In [6]:

X_gender = X_gender / 255.0
X_emotion = X_emotion / 255.0

le_gender = LabelEncoder()
y_gender = le_gender.fit_transform(y_gender)

le_emotion = LabelEncoder()
y_emotion = le_emotion.fit_transform(y_emotion)


### Crear modelo

In [7]:
def build_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

### Aplicar kfold y Entrenar

In [8]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

X = X_emotion  # o X_gender
y = y_emotion  # o y_gender

fold = 1
for train_idx, val_idx in kf.split(X):
    print(f"\n📂 Fold {fold}")
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = build_cnn(X.shape[1:], len(np.unique(y)))
    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)
    
    fold += 1


📂 Fold 1


/home/nicolo/anaconda3/envs/tf_gpu/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1763059413.213103   29079 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2140 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-11-13 18:43:35.447652: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 637452288 exceeds 10% of free system memory.
2025-11-13 18:43:35.944823: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 637452288 exceeds 10% of free system memory.


Epoch 1/10


2025-11-13 18:43:37.164818: I external/local_xla/xla/service/service.cc:163] XLA service 0x778bf400bbd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-13 18:43:37.164843: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-11-13 18:43:37.194358: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-13 18:43:37.412767: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91501
2025-11-13 18:43:37.513064: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-13 18:43:38.

 38/721 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2211 - loss: 1.9079

...

721/721 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2679 - loss: 1.7899

2025-11-13 18:43:46.915326: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 159390720 exceeds 10% of free system memory.
2025-11-13 18:43:47.045320: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 159390720 exceeds 10% of free system memory.


721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.3140 - loss: 1.7096 - val_accuracy: 0.4156 - val_loss: 1.5311
Epoch 2/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4165 - loss: 1.5189 - val_accuracy: 0.4536 - val_loss: 1.4183
Epoch 3/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4530 - loss: 1.4210 - val_accuracy: 0.4796 - val_loss: 1.3785
Epoch 4/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4758 - loss: 1.3578 - val_accuracy: 0.4794 - val_loss: 1.3500
Epoch 5/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5045 - loss: 1.2998 - val_accuracy: 0.4942 - val_loss: 1.3291
Epoch 6/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5211 - loss: 1.2477 - val_accuracy: 0.5074 - val_loss: 1.2856
Epoch 7/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5443 - loss: 1.1845 - val_accuracy: 0.5023 - val_loss: 1.2939
Epoch 8/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5638 - loss: 1.1383 - val_accuracy: 0.5110 - va

2025-11-13 18:44:21.560955: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 637479936 exceeds 10% of free system memory.


Epoch 1/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.3169 - loss: 1.7117 - val_accuracy: 0.4244 - val_loss: 1.5317
Epoch 2/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4164 - loss: 1.5107 - val_accuracy: 0.4603 - val_loss: 1.4072
Epoch 3/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4563 - loss: 1.4067 - val_accuracy: 0.4828 - val_loss: 1.3600
Epoch 4/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4899 - loss: 1.3315 - val_accuracy: 0.4988 - val_loss: 1.3093
Epoch 5/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5145 - loss: 1.2732 - val_accuracy: 0.5073 - val_loss: 1.2891
Epoch 6/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5383 - loss: 1.2110 - val_accuracy: 0.5144 - val_loss: 1.2867
Epoch 7/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5571 - loss: 1.1563 - val_accuracy: 0.5213 - val_loss: 1.2660
Epoch 8/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5771 - loss: 1.1066 - val_accuracy: 

### Guardar Modelo

In [9]:
model.save("cnn_emotion_model.h5")

### Carga de Modelo
El paso de guardar el modelo anterior no es necesario, pero en mi caso si ya que en macOS la camara tiene la tendencia de crashear el kernel de python y es necesario recargar todo y si tuviese que reentrenar el modelo en cada ocasión no podría hacer una defensa a tiempo.

In [ ]:
model = load_model("cnn_emotion_model.h5")

### Carga de filtros


In [11]:
def filtro_happy(frame):
    # Aumentar brillo
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    v = cv2.add(v, 50)  # subir brillo
    v = np.clip(v, 0, 255)
    final_hsv = cv2.merge((h,s,v))
    return cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

def filtro_sad(frame):
    # Tonos azules (enfriar imagen)
    frame = cv2.addWeighted(frame, 0.7, np.zeros_like(frame), 0.3, 0)
    frame[:,:,0] = cv2.add(frame[:,:,0], 50)  # azul más intenso
    return frame

def filtro_angry(frame):
    # Tonos rojos
    frame = cv2.addWeighted(frame, 0.7, np.zeros_like(frame), 0.3, 0)
    frame[:,:,2] = cv2.add(frame[:,:,2], 50)  # rojo más intenso
    return frame


### Resolución

In [ ]:
labels = le_emotion.classes_  # ej. ['angry', 'happy', 'sad', ...]

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (48,48)) / 255.0
    img_resized = np.expand_dims(img_resized, axis=0)

    preds = model.predict(img_resized)
    label = labels[np.argmax(preds)]
    confidence = np.max(preds)

    # Aplicar filtro según emoción
    if label == "happy":
        frame = filtro_happy(frame)
    elif label == "sad":
        frame = filtro_sad(frame)
    elif label == "angry":
        frame = filtro_angry(frame)
    # etc.

    cv2.putText(frame, f"{label} ({confidence*100:.1f}%)", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Emotion Filters", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━